In [32]:
import pandas as pd
from rdkit import Chem

In [33]:
ROOT = "s3://sg-datasources-v2"

In [34]:
df_all = pd.read_parquet(f"{ROOT}/pre_filtered/measurements/relation.parquet/")

In [35]:
# Create a dict whose keys are the CP targets and values are dataframes

In [36]:
df_all.type.value_counts()

IC50                    7713181
Ki                      1748527
Inhibition               927259
EC50                     493315
Kd                       350907
InhibitionDiffAt10um     306893
InhibitionDiffAt1um      306893
LogS                     224107
LogD                      30131
Papp                      10529
Permeability               1792
MS_Human                     67
MS_Mouse                     67
fup_Human                    67
fup_Mouse                    67
Name: type, dtype: int64

In [37]:
cp_data_dict = {}

In [38]:
cp_data_dict['LogS'] = df_all[df_all['type'] == 'LogS']

cp_data_dict['LogD'] = df_all[df_all['type'] == 'LogD']

cp_data_dict['Papp'] = df_all[df_all['type'] == 'Papp']


In [39]:
df_all[df_all['type'] == 'IC50']

,id,relation,type,source,version,filter
493315,3155,=,IC50,bindingdb,v2022m4,identity
493316,3199,=,IC50,bindingdb,v2022m4,identity
493317,3327,=,IC50,bindingdb,v2022m4,identity
493318,3391,=,IC50,bindingdb,v2022m4,identity
493319,3800,=,IC50,bindingdb,v2022m4,identity
...,...,...,...,...,...,...
8206491,532575944705,>,IC50,sg_cpi_experiment,v20221231,identity
8206492,541165879296,>,IC50,sg_cpi_experiment,v20221231,identity
8206493,541165879297,>,IC50,sg_cpi_experiment,v20221231,identity
8206494,541165879298,>,IC50,sg_cpi_experiment,v20221231,identity


In [40]:
# Getting the IC50 related data

In [41]:
TYPE = 'IC50'

In [42]:
%%time
# Get y_value scalar
df = pd.read_parquet(
    f"{ROOT}/pre_filtered/measurements/value.parquet/",
    filters=[("type", "=", TYPE)]
)

CPU times: user 3.44 s, sys: 990 ms, total: 4.43 s
Wall time: 53 s


In [43]:
%%time
# Cf.in case of reading all
df = df.merge(
    pd.read_parquet(
        f"{ROOT}/pre_filtered/measurements/relation.parquet/",
        filters=[("type", "=", TYPE)],
    ),
    on=["id", "type", "source", "version", "filter"],
)

CPU times: user 8.09 s, sys: 1.6 s, total: 9.69 s
Wall time: 55.9 s


In [44]:
# Cf.in case of reading all
df = df.merge(
    pd.read_parquet(
        f"{ROOT}/pre_filtered/measurements/protein_id.parquet/",
        filters=[("type", "=", TYPE)],
    ),
    on=["id", "type", "source", "version", "filter"],
)

In [45]:
df.head()

,id,value,type,source,version,filter,relation,protein_id
0,3155,7700.0,IC50,bindingdb,v2022m4,identity,=,P68403
1,3199,14500.0,IC50,bindingdb,v2022m4,identity,=,P00517
2,3327,2000.0,IC50,bindingdb,v2022m4,identity,=,P00517
3,3391,3900.0,IC50,bindingdb,v2022m4,identity,=,P00517
4,3800,19000.0,IC50,bindingdb,v2022m4,identity,=,P04409


In [46]:
UNIPROT_IDS = {
    "hERG": "Q12809",
    "CYP1A2": "P05177",
    "CYP2B6": "P20813",
    "CYP2C8": "P10632",
    "CYP2C19": "P33261",
    "CYP2C9": "P11712",
    "CYP2D6": "P10635",
    "CYP3A4": "P08684",
}

In [47]:
for k in UNIPROT_IDS.keys():
    cp_data_dict[k] = df[df.protein_id == UNIPROT_IDS[k]]
    

In [50]:
for k in cp_data_dict.keys():
    print(k,len(cp_data_dict[k]))

LogS 224107
LogD 30131
Papp 10529
hERG 42620
CYP1A2 10906
CYP2B6 1873
CYP2C8 2880
CYP2C19 11587
CYP2C9 19097
CYP2D6 23849
CYP3A4 37555


In [52]:
cp_data_dict['LogS'].source.value_counts()

kinect_solcuration      164273
chembl_solcuration       34836
acs_jcim_0c00568         10475
aqsoldb                   9982
phys_solcuration          2001
aqua_solcuration          1310
esol_solcuration          1121
sg_cp_experiment           109
bindingdb                    0
minf_201000118               0
acs_jcim_5b00642             0
deepchemlipo                 0
chembl                       0
herg_central                 0
sg_cpi_experiment            0
pubchem                      0
kinase_knowledgebase         0
gostar                       0
peerj_1405                   0
Name: source, dtype: int64

In [70]:
def fetch_molbinary_data(data_dict,type):
    
    tmp = data_dict[type].merge(pd.read_parquet(
        f"{ROOT}/pre_filtered/measurements/compound_id.parquet/",
        filters=[("type", "=", type)],
    ),on = ["id", "type", "source", "version", "filter"])

    tmp1 = tmp[["source", "version"]].drop_duplicates()

    versions = dict(zip(tmp['source'],tmp['version']))

    c_filters = [[("source", "=", s), ("version", "=", v)] for s, v in versions.items()]

    c_df = pd.concat([pd.read_parquet(
        f"{ROOT}/pre_filtered/compounds/molbinary.parquet/",
        filters=filters) for filters in c_filters])

    tmp2 = tmp.merge(c_df, left_on=["source", "version", "compound_id", "filter"],
     right_on=["source", "version", "id", "filter"], 
     suffixes=["", "_c"])
     
    return tmp2     

In [72]:
cp_data_dict_mb = {}

for k in cp_data_dict.keys():
    try: 
       cp_data_dict_mb[k] = fetch_molbinary_data(cp_data_dict,k)
    except:
        print(k)
        cp_data_dict_mb[k] = None   

Papp
hERG
CYP1A2
CYP2B6
CYP2C8
CYP2C19
CYP2C9
CYP2D6
CYP3A4


In [77]:
tmp = pd.read_parquet(
        f"{ROOT}/pre_filtered/measurements/compound_id.parquet/",
        filters=[("type", "=", 'Papp')],
    )


In [78]:
tmp1 = cp_data_dict['Papp'].merge(tmp, on = ["id", "type", "source", "version", "filter"])

In [55]:
tmp = cp_data_dict['LogS'].merge(pd.read_parquet(
        f"{ROOT}/pre_filtered/measurements/compound_id.parquet/",
        filters=[("type", "=", 'LogS')],
    ),on = ["id", "type", "source", "version", "filter"])

In [80]:
len(tmp)

tmp1 = tmp[["source", "version"]].drop_duplicates()

versions = dict(zip(tmp['source'],tmp['version']))

In [84]:
c_filters = [[("source", "=", s), ("version", "=", v)] for s, v in versions.items()]


In [85]:
c_filters

[[('source', '=', 'acs_jcim_0c00568'), ('version', '=', 'v20200817')],
 [('source', '=', 'acs_jcim_5b00642'), ('version', '=', 'v201606')],
 [('source', '=', 'chembl'), ('version', '=', 'v29')],
 [('source', '=', 'minf_201000118'), ('version', '=', 'v20110331')],
 [('source', '=', 'peerj_1405'), ('version', '=', 'v201511')],
 [('source', '=', 'sg_cp_experiment'), ('version', '=', 'v20220823_1')]]

In [88]:
for i,f in enumerate(c_filters):
    print(f)
    t = pd.read_parquet(f"{ROOT}/pre_filtered/compounds/molbinary.parquet/",filters=f)
    t.to_parquet('{}.parquet'.format(f))

[('source', '=', 'acs_jcim_0c00568'), ('version', '=', 'v20200817')]
[('source', '=', 'acs_jcim_5b00642'), ('version', '=', 'v201606')]
[('source', '=', 'chembl'), ('version', '=', 'v29')]


FSTimeoutError: 

In [83]:
c_filters = [[("source", "=", s), ("version", "=", v)] for s, v in versions.items()]

c_df = pd.concat([pd.read_parquet(
        f"{ROOT}/pre_filtered/compounds/molbinary.parquet/",
        filters=filters) for filters in c_filters])

FSTimeoutError: 

In [61]:
tmp1 = tmp[["source", "version"]].drop_duplicates()

versions = dict(zip(tmp['source'],tmp['version']))

In [63]:
c_filters = [
    [("source", "=", s), ("version", "=", v)] for s, v in versions.items()
]
c_filters

[[('source', '=', 'acs_jcim_0c00568'), ('version', '=', 'v20200817')],
 [('source', '=', 'aqsoldb'), ('version', '=', 'v2019')],
 [('source', '=', 'aqua_solcuration'), ('version', '=', 'v20200701')],
 [('source', '=', 'chembl_solcuration'), ('version', '=', 'v20200701')],
 [('source', '=', 'esol_solcuration'), ('version', '=', 'v20200701')],
 [('source', '=', 'kinect_solcuration'), ('version', '=', 'v20200701')],
 [('source', '=', 'phys_solcuration'), ('version', '=', 'v20200701')],
 [('source', '=', 'sg_cp_experiment'), ('version', '=', 'v20220823_1')]]

In [65]:
%%time
# Get compounds of specific sources utilizing Hive
c_df = pd.concat([pd.read_parquet(
        f"{ROOT}/pre_filtered/compounds/molbinary.parquet/",
        filters=filters
    )
    for filters in c_filters
])

CPU times: user 27.1 s, sys: 1.72 s, total: 28.8 s
Wall time: 5min 37s


In [67]:
tmp2 = tmp.merge(
    c_df,
    left_on=["source", "version", "compound_id", "filter"],
    right_on=["source", "version", "id", "filter"],
    suffixes=["", "_c"],
)